In [2]:
#read the input data
with open('input.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()

In [7]:
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [5]:
print("length of dataset in characters:", len(text))

length of dataset in characters: 1115394


In [7]:
#here all the unique chracters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [8]:
#create a mapping from characters to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]  # take a string, output a list of integers
decode = lambda x: [''.join(itos[i] for i in x)] #take a sequence of integers, output a string

print(encode("Hi there!"))
print(decode(encode("Hi there!")))

[20, 47, 1, 58, 46, 43, 56, 43, 2]
['Hi there!']


In [9]:
#enocde the whole text data
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [10]:
#Let's split the data into training and validation datasets
split = int(0.9 * len(data))
train_data = data[:split]
val_data = data[split:]

In [15]:
context_length = 8 # the length of input to the model 


# in this splitting, we hace considered that for a chunk size between 1 and context_length
# in the x (input), the subsequent sequence of the same length will behave as y (target) 
# which will essentially tell the model the following

# in x = [18, 3, 34, 25, 63, 6], 
# [18] -> [3]
# [18, 3] -> [34]
# [18, 3, 34] -> [25]
# and so on


x = train_data[:context_length]
y = train_data[1:context_length+1]
for t in range(context_length):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} -> output is {target}")

when input is tensor([18]) -> output is 47
when input is tensor([18, 47]) -> output is 56
when input is tensor([18, 47, 56]) -> output is 57
when input is tensor([18, 47, 56, 57]) -> output is 58
when input is tensor([18, 47, 56, 57, 58]) -> output is 1
when input is tensor([18, 47, 56, 57, 58,  1]) -> output is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) -> output is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) -> output is 58


In [17]:
len(train_data)

1003854

In [20]:
#get the blocks in batches
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size + 1] for i in ix])

    return x, y

xb, yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('------')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")
    print
    print("---------")
print("==========")

inputs:
torch.Size([4, 8])
tensor([[52, 53, 52, 43,  1, 57, 53,  8],
        [24, 13, 10,  0, 14, 43, 41, 39],
        [ 6,  0, 21,  1, 42, 53,  1, 40],
        [ 1, 39, 51,  1, 53, 52, 43,  1]])
targets:
torch.Size([4, 8])
tensor([[53, 52, 43,  1, 57, 53,  8,  1],
        [13, 10,  0, 14, 43, 41, 39, 59],
        [ 0, 21,  1, 42, 53,  1, 40, 43],
        [39, 51,  1, 53, 52, 43,  1, 58]])
------
when input is [52] the target: 53
when input is [52, 53] the target: 52
when input is [52, 53, 52] the target: 43
when input is [52, 53, 52, 43] the target: 1
when input is [52, 53, 52, 43, 1] the target: 57
when input is [52, 53, 52, 43, 1, 57] the target: 53
when input is [52, 53, 52, 43, 1, 57, 53] the target: 8
when input is [52, 53, 52, 43, 1, 57, 53, 8] the target: 1
---------
when input is [24] the target: 13
when input is [24, 13] the target: 10
when input is [24, 13, 10] the target: 0
when input is [24, 13, 10, 0] the target: 14
when input is [24, 13, 10, 0, 14] the target: 43
when in

Let's now first pass our data through the bigram language model to get a baseline performance 

In [59]:
import torch
from torch import nn
import torch.nn.functional as F
torch.manual_seed(346)
class BiGram(nn.Module):
    def __init__(self, vocab_size : int):
        super().__init__()
        self.vocab_size = vocab_size

        #create a lookup table for the embeddings of the characters in the
        #vocabulary so that they can be easily accessed
        self.token_embeddings_table = nn.Embedding(vocab_size, vocab_size)

    #idx and targets are both (B, T) tensor of integers
    def forward(self, idx : torch.Tensor, targets = None):


        #B = Batch, T = Time; basically the idx, C = Channels; dimensions of embeddings
        logits = self.token_embeddings_table(idx) # shape (B, T, C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = (F.cross_entropy(logits, targets))

        return logits, loss
    
    def generate(self, idx : torch.Tensor, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)

            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)

        return idx
    
    


In [60]:
m = BiGram(vocab_size)

In [61]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=0.1)
for steps in range(1000):
    #sample a batch of data
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
    if steps % 100 == 0:
        print(loss.item())
    


4.927609443664551
2.5466856956481934
2.60264253616333
2.4136760234832764
2.226475477218628
2.5667262077331543
2.3731019496917725
3.078883647918701
2.605280637741089
2.5369832515716553


In [41]:
#logits, loss = m(xb, yb)
#print(loss)
#print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens = 100)[0].tolist()))

["\nT:\n\nTY!\nFowa s y'dy orach h ambe m.\nT:\nCLejofougr,\nI' buthitithalenougangucomitas wichend heald!\nAum"]


### The mathematical trick in self-attention

Multiplying a matrix with a lower trainagular matrix gives the self-attentionfor that matrix

In [64]:
a = torch.ones(size = (3, 3))
b = torch.Tensor([[1, 2, 3],
                 [4, 5, 6]])
c = torch.tril(torch.ones((3, 2)))
c @ b # is the self - attention

tensor([[1., 2., 3.],
        [5., 7., 9.],
        [5., 7., 9.]])

In [68]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2 #batch, time, channels
x = torch.randn(B, T, C)

In [71]:
#continued average along each of the rows
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, )

In [79]:
#writing it using matrix multiplication
wei = torch.tril(torch.ones(T, T))
wei = wei/wei.sum(1, keepdim=True) #this will help in broadcasting
xbow2 = wei @ x # (B, T, T) @ (B, T, C) -------> (B, T, C)
xbow2.shape

torch.Size([4, 8, 2])

In [81]:
import torch.nn.functional as F
#using softmax to write this
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x


### Writing the self-attention layer

In [86]:
import torch.nn.functional as F
import torch.nn as nn
torch.manual_seed(1273)

# B = Batch 
# T = Time
# C = Channels or embeeding size
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

#the initialization with zeros of weight matrix is not necessary. THse values represent the affinity
#of each of the token with each other. The values of this wei matrix depends on the \
#dataset. Self attention solves this initialization problem

# -----SELF ATTENTION------#
# We make three information vectors come out of each input, key, query and value. Key and query can be 
# understood in the same sense as a youtube search task, where what we put in the search bar is the query
# the information available with the platform is key and their dot product will tell you how mush similarity
# present between the query and the key. 
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)
q = query(x)
wei = q @ k.transpose(-2, -1) #(B, T, 16) @ (B, 16, T) -> (B, T, T)


tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)

#The value term is used to find the similarity between what is generated by the mechanism (wei) 
#and what is expected based on the input.
v = value(x)
out = wei @ v



torch.Size([4, 8, 32])
